**Digit Recognizer**

This competition involves training a neural network to be able to recognize handwritten digits.

The MNIST ("Modified National Institute of Standards and Technology") dataset contains a large number of handwritten digits, and is often used to train neural networks for vision-based tasks.

More information can be found on the competition page.
https://www.kaggle.com/c/digit-recognizer/overview/description

In [1]:
# Let's keep all the imports we need here
# data analysis and wrangling
import pandas as pd
import numpy as np
# preprocessing
from keras.utils.np_utils import to_categorical
# model
from tensorflow.keras.models import Sequential
# layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D

Using TensorFlow backend.


In [2]:
# Suppress future warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# Let's read in the data first
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

Now that we've loaded in the data, we first want to see what exactly we will be working with.

In [4]:
print(train.shape)
train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


From this we see that the first column is the label showing which digit the row represents, followed by 784 pixels.
We also see that the training dataset contains 42,000 training instances.

In [5]:
# We know this is a multiclass classification problem, but let's see how many classes there are
train.label.unique()

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6])

We have training data for the digits 0-9, giving as 10 total classes.



**Data preprocessing**

Right now we have a row of pixels, but what they really represent is a square image. We need to do some preprocessing to get it into that shape. The [competition homepage](https://www.kaggle.com/c/digit-recognizer/data) explains how.

> Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.
>
>The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.
>
>Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).
>
>For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.
>
>Visually, if we omit the "pixel" prefix, the pixels make up the image like this:
```
000 001 002 003 ... 026 027
028 029 030 031 ... 054 055
056 057 058 059 ... 082 083
 |   |   |   |  ...  |   |
728 729 730 731 ... 754 755
756 757 758 759 ... 782 783 
```
>The test data set, (test.csv), is the same as the training set, except that it does not contain the "label" column.

In [6]:
# Let's separate the pixels from the labels
X_train = (train.iloc[:,1:].values).astype('float32')
y_train = train.iloc[:,0].values.astype('int32')
# The test set has no labels
X_test = test.values.astype('float32')

In [7]:
# Normalize the data so the range goes from 0-255, to 0-1
X_train = X_train / 255.0
X_test = X_test / 255.0

In [8]:
# Reshape the array so each image is 28 by 28 by 1
# The last dimension is the pixel value
X_train = X_train.reshape(-1, 28, 28,1)
X_test = X_test.reshape(-1,28,28,1)

**One Hot Encondings**

Right now our labels are ordinal values. If left as is, our model would think we are trying to score each image from 0 to 9. 
Since that is not the case, and we want the model to classify each image as one of ten classes, we will convert our labels using one hot enconding. 
This will create a separate column for each class, 
and each image will have zeroes in every column, except for one, 
e.g., an image displaying the digit 0, would have zeroes in every column except the first one.

In [9]:
y_train = to_categorical(y_train)

**Models**

Before we create any models, we need to split our training data into a training and validation set.

This is done so we can compare the performance of different models.

In [10]:
# Set aside some data for validation
from sklearn.model_selection import train_test_split
X = X_train
y = y_train
# Validation set will be 10% of the training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

We will start by creating a small, simple network to compare our later networks against.

In [11]:
model = Sequential()
model.add(Conv2D(28, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(28, 28, 1)))

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=64,
          epochs=6,
          validation_data=(X_val, y_val))

Instructions for updating:
Colocations handled automatically by placer.
Train on 37800 samples, validate on 4200 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/6
37800/37800 [==============================] - 96s 3ms/sample - loss: 0.1404 - acc: 0.9569 - val_loss: 0.0665 - val_acc: 0.9786
Epoch 2/6
37800/37800 [==============================] - 97s 3ms/sample - loss: 0.0422 - acc: 0.9868 - val_loss: 0.0410 - val_acc: 0.9871
Epoch 3/6
37800/37800 [==============================] - 95s 3ms/sample - loss: 0.0272 - acc: 0.9912 - val_loss: 0.0507 - val_acc: 0.9840
Epoch 4/6
37800/37800 [==============================] - 97s 3ms/sample - loss: 0.0179 - acc: 0.9938 - val_loss: 0.0345 - val_acc: 0.9898
Epoch 5/6
37800/37800 [==============================] - 96s 3ms/sample - loss: 0.0121 - acc: 0.9960 - val_loss: 0.0367 - val_acc: 0.9895
Epoch 6/6
37800/37800 [==============================] - 96s 3ms/sample - loss: 0.0124 - acc: 0.9959 - val_loss: 0.0672 - val_acc: 0.9800


This gives us ~98% accuracy accuracy. Taking into account that individual runs can vary by about 0.5%, we can only expect some minimal improvements.

Ideally we would want to get at least 99% validation accuracy.

For the next network we will add a pooling layer as well as a dropout layer.

In [12]:
model2 = Sequential()
model2.add(Conv2D(28, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(28, 28, 1)))

model2.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(10, activation='softmax'))
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model2.fit(X_train, y_train,
          batch_size=64,
          epochs=6,
          validation_data=(X_val, y_val))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 37800 samples, validate on 4200 samples
Epoch 1/6
37800/37800 [==============================] - 84s 2ms/sample - loss: 0.2067 - acc: 0.9374 - val_loss: 0.0674 - val_acc: 0.9781
Epoch 2/6
37800/37800 [==============================] - 84s 2ms/sample - loss: 0.0729 - acc: 0.9779 - val_loss: 0.0400 - val_acc: 0.9876
Epoch 3/6
37800/37800 [==============================] - 83s 2ms/sample - loss: 0.0513 - acc: 0.9844 - val_loss: 0.0384 - val_acc: 0.9886
Epoch 4/6
37800/37800 [==============================] - 84s 2ms/sample - loss: 0.0425 - acc: 0.9865 - val_loss: 0.0349 - val_acc: 0.9900
Epoch 5/6
37800/37800 [==============================] - 83s 2ms/sample - loss: 0.0345 - acc: 0.9888 - val_loss: 0.0291 - val_acc: 0.9910
Epoch 6/6
37800/37800 [==============================] - 84s 2ms/sample - loss: 0.0296 - acc: 0.9910 - val_loss: 0.0316 - val_acc: 0.9914


This gives us ~99% accuracy, which is good enough for our purposes.
This model is still on the simple side, and while a more complex model might give us better accuracy, I don't think what would be at most 1% more accuracy would be worth the drop in performance from such a model.

The next step is to create the submission file to be submitted to the competition.

In [13]:
pred = model2.predict_classes(X_test, verbose=0)
submission = pd.DataFrame({"ImageId": list(range(1,len(pred)+1)),
                         "Label": pred})
submission.to_csv("submission.csv", index=False, header=True)